In [1]:
from typing import Callable


class Tool:
    """
    A class representing a reusable piece of code (Tool).

    Attributes:
        name (str): Name of the tool.
        description (str): A textual description of what the tool does.
        func (callable): The function this tool wraps.
        arguments (list): A list of arguments.
        outputs (str or list): The return type(s) of the wrapped function.
    """
    def __init__(self,
                 name: str,
                 description: str,
                 func: Callable,
                 arguments: list,
                 outputs: str):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = arguments
        self.outputs = outputs

    def to_string(self) -> str:
        """
        Return a string representation of the tool,
        including its name, description, arguments, and outputs.
        """
        args_str = ", ".join([
            f"{arg_name}: {arg_type}" for arg_name, arg_type in self.arguments
        ])

        return (
            f"Tool Name: {self.name},"
            f" Description: {self.description},"
            f" Arguments: {args_str},"
            f" Outputs: {self.outputs}"
        )

    def __call__(self, *args, **kwargs):
        """
        Invoke the underlying function (callable) with provided arguments.
        """
        return self.func(*args, **kwargs)
    

def multiply_numbers(a: int, b:int) -> int:
    return a*b

add_tool= Tool(
    name="multiply_numbers",
    description="multiplies 2 numbers",
    func=multiply_numbers,
    arguments=[("a", "int"), ("b", int)],
    outputs=int
)

print (add_tool.to_string())

result= add_tool(5,7)
print("Result: ",result)

Tool Name: multiply_numbers, Description: multiplies 2 numbers, Arguments: a: int, b: <class 'int'>, Outputs: <class 'int'>
Result:  35


In [3]:
from typing import Callable, List, Tuple, Any, Dict


class Tool:
    """Simple wrapper around a callable with metadata."""
    def __init__(self,
                 name: str,
                 description: str,
                 func: Callable,
                 arguments: List[Tuple[str, str]],
                 outputs: str):
        self.name = name
        self.description = description
        self.func = func
        # ensure argument types are strings for safe display
        self.arguments = [(n, str(t)) for n, t in arguments]
        self.outputs = str(outputs)

    def to_string(self) -> str:
        args_str = ", ".join(f"{n}: {t}" for n, t in self.arguments)
        return (f"Tool Name: {self.name}, Description: {self.description}, "
                f"Arguments: {args_str}, Outputs: {self.outputs}")

    def __call__(self, *args, **kwargs) -> Any:
        return self.func(*args, **kwargs)


class ToolRegistry:
    """Register tools and execute them by name."""
    def __init__(self):
        self._tools: Dict[str, Tool] = {}

    def register(self, tool: Tool) -> None:
        self._tools[tool.name] = tool

    def run(self, tool_name: str, *args, **kwargs) -> Any:
        if tool_name not in self._tools:
            raise ValueError(f"Tool '{tool_name}' is not registered.")
        return self._tools[tool_name](*args, **kwargs)

    def list_tools(self) -> List[str]:
        return list(self._tools.keys())


class Agent:
    """Agent that can execute a subset of registered tools."""
    def __init__(self, name: str, allowed_tools: List[str], registry: ToolRegistry):
        self.name = name
        self.allowed_tools = set(allowed_tools)
        self.registry = registry

    def can_run(self, tool_name: str) -> bool:
        return tool_name in self.allowed_tools

    def run_tool(self, tool_name: str, *args, **kwargs) -> Any:
        if not self.can_run(tool_name):
            raise PermissionError(f"Agent '{self.name}' not allowed to run '{tool_name}'.")
        print(f"[Agent {self.name}] Running tool '{tool_name}' with args {args} {kwargs}")
        result = self.registry.run(tool_name, *args, **kwargs)
        print(f"[Agent {self.name}] Result: {result}")
        return result

    def receive_task(self, task: dict) -> Any:
        """
        Task format: {"tool": "tool_name", "args": [...], "kwargs": {...}}
        """
        tool = task.get("tool")
        args = task.get("args", [])
        kwargs = task.get("kwargs", {})
        return self.run_tool(tool, *args, **kwargs)


class Coordinator:
    """Simple coordinator that assigns tasks to agents in round-robin fashion."""
    def __init__(self, agents: List[Agent]):
        self.agents = agents
        self._next = 0

    def assign(self, task: dict) -> Any:
        # try to find an agent that can run the tool (round-robin start)
        n = len(self.agents)
        for i in range(n):
            idx = (self._next + i) % n
            agent = self.agents[idx]
            if agent.can_run(task["tool"]):
                self._next = (idx + 1) % n
                print(f"[Coordinator] Assigning task {task} -> Agent '{agent.name}'")
                return agent.receive_task(task)
        raise RuntimeError(f"No available agent can run tool '{task['tool']}'.")


# -------------------------
# Example tools and agents
# -------------------------

# small functions
def add(a: int, b: int) -> int:
    return a + b

def multiply(a: int, b: int) -> int:
    return a * b

def greet(name: str) -> str:
    return f"Hello, {name}!"

# create tools
add_tool = Tool("add", "Add two integers", add, [("a", "int"), ("b", "int")], "int")
mul_tool = Tool("multiply", "Multiply two integers", multiply, [("a", "int"), ("b", "int")], "int")
greet_tool = Tool("greet", "Greet a person", greet, [("name", "str")], "str")

# registry and registration
registry = ToolRegistry()
for t in (add_tool, mul_tool, greet_tool):
    registry.register(t)

# agents with different capabilities
agent_adder = Agent("AdderAgent", allowed_tools=["add"], registry=registry)
agent_multiplier = Agent("MultiplierAgent", allowed_tools=["multiply"], registry=registry)
agent_general = Agent("GeneralAgent", allowed_tools=["add", "multiply", "greet"], registry=registry)

# coordinator
coord = Coordinator([agent_adder, agent_multiplier, agent_general])

# tasks
tasks = [
    {"tool": "add", "args": [2, 3]},
    {"tool": "multiply", "args": [4, 5]},
    {"tool": "greet", "args": ["HelloWorld"]},
    {"tool": "add", "args": [10, 20]},
]

# run tasks
for t in tasks:
    output = coord.assign(t)
    print("-> Final output:", output)
    print("-" * 40)


[Coordinator] Assigning task {'tool': 'add', 'args': [2, 3]} -> Agent 'AdderAgent'
[Agent AdderAgent] Running tool 'add' with args (2, 3) {}
[Agent AdderAgent] Result: 5
-> Final output: 5
----------------------------------------
[Coordinator] Assigning task {'tool': 'multiply', 'args': [4, 5]} -> Agent 'MultiplierAgent'
[Agent MultiplierAgent] Running tool 'multiply' with args (4, 5) {}
[Agent MultiplierAgent] Result: 20
-> Final output: 20
----------------------------------------
[Coordinator] Assigning task {'tool': 'greet', 'args': ['HelloWorld']} -> Agent 'GeneralAgent'
[Agent GeneralAgent] Running tool 'greet' with args ('HelloWorld',) {}
[Agent GeneralAgent] Result: Hello, HelloWorld!
-> Final output: Hello, HelloWorld!
----------------------------------------
[Coordinator] Assigning task {'tool': 'add', 'args': [10, 20]} -> Agent 'AdderAgent'
[Agent AdderAgent] Running tool 'add' with args (10, 20) {}
[Agent AdderAgent] Result: 30
-> Final output: 30
---------------------------